In [1]:
%pip install pandas
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sleeperpy import User

In [3]:
matchups = pd.read_parquet('s3://datalakestack-datalakebucket0256ea8e-jlou2zsfqxmr/sleeper/matchups/')

In [4]:
owner_list = []
for owner_id in matchups['owner_id'].unique():
    owner_list.append(User.get_user(owner_id))
owners = pd.DataFrame(owner_list)
matchups = pd.merge(matchups, owners, left_on='owner_id', right_on='user_id', how='inner')

In [38]:
matchups.columns

7

In [57]:
def calculate_points_against(row):
    points_against = matchups.loc[matchups['year'] == row['year']].loc[matchups['week'] == row['week']].loc[matchups['matchup_id'] == row['matchup_id']].loc[matchups['owner_id'] != row['owner_id']]['points'].values
    if len(points_against) == 1:
        return points_against[0]
    else:
        return 0
matchups['points_against'] = matchups.apply(calculate_points_against, axis=1)

In [59]:
matchups.loc[matchups['type'] == 'regular'].loc[matchups['year'] == 2023].groupby(['username'])['points_against'].sum()

username
acarlough        1468.45
almostwinning    1519.63
bigoledoinks     1385.72
callentrail      1578.39
coolguytrav      1613.30
cramseyio        1563.58
exoentropy       1467.21
jyarrow          1535.64
lrich91          1478.36
markhambone      1378.42
mattkman         1691.05
novaprice        1473.96
Name: points_against, dtype: float64

In [8]:
matchups.loc[matchups['type'] == 'playoff'].groupby(['username'])['points'].sum()

username
acarlough        1363.15
almostwinning    1441.71
bigoledoinks     1474.59
callentrail      1589.59
coolguytrav      1472.75
cramseyio        1417.09
exoentropy       1582.88
jyarrow          1356.66
lrich91          1404.21
markhambone      1631.08
mattkman         1676.75
novaprice        1593.44
Name: points, dtype: float64